In [1]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

omopcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x55a650888c10>


In [2]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
from dask.distributed import Client

In [4]:
df_DEATH = pd.read_sql('select * from `DEATH`', con=conn)

/tmp/ipykernel_131227/2509977731.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_DEATH = pd.read_sql('select * from `DEATH`', con=conn)


In [5]:
print('Length - DEATH: ', len(df_DEATH))

Length - DEATH:  0


In [6]:
print(df_DEATH.head())

Empty DataFrame
Columns: [person_id, death_date, death_datetime, death_type_concept_id, cause_concept_id, cause_source_value, cause_source_concept_id]
Index: []


In [7]:
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

mimicivconn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='mimiciv')

mimicivcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x55a650888c10>


In [26]:
df_admissions_mimiciv = pd.read_sql('select subject_id, deathtime, hospital_expire_flag  from admissions', con=mimicivconn)
print(df_admissions_mimiciv.head()) 

/tmp/ipykernel_131227/96515154.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_admissions_mimiciv = pd.read_sql('select subject_id, deathtime, hospital_expire_flag  from admissions', con=mimicivconn)


   subject_id deathtime  hospital_expire_flag
0    10467237       NaT                     0
1    16925328       NaT                     0
2    19430048       NaT                     0
3    18910522       NaT                     0
4    11146739       NaT                     0


In [27]:
ddf_admissions_mimiciv = dd.from_pandas(df_admissions_mimiciv).compute()

In [28]:
df_admissions_mimiciv = ''

In [29]:
print(ddf_admissions_mimiciv)

        subject_id deathtime  hospital_expire_flag
0         10467237       NaT                     0
1         16925328       NaT                     0
2         19430048       NaT                     0
3         18910522       NaT                     0
4         11146739       NaT                     0
...            ...       ...                   ...
431226    10382924       NaT                     0
431227    11326722       NaT                     0
431228    12133002       NaT                     0
431229    16572164       NaT                     0
431230    16549335       NaT                     0

[431231 rows x 3 columns]


In [30]:
df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group from patients', con=mimicivconn)
print(df_patients_mimiciv.head()) 

   subject_id  anchor_year anchor_year_group
0    10000032         2180       2014 - 2016
1    10000048         2126       2008 - 2010
2    10000068         2160       2008 - 2010
3    10000084         2160       2017 - 2019
4    10000102         2136       2008 - 2010


/tmp/ipykernel_131227/587641350.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_patients_mimiciv = pd.read_sql('select subject_id, anchor_year, anchor_year_group from patients', con=mimicivconn)


In [31]:
ddf_patients_mimiciv = dd.from_pandas(df_patients_mimiciv).compute()

In [32]:
df_patients_mimiciv = ''

In [33]:
print(ddf_patients_mimiciv.head())

   subject_id  anchor_year anchor_year_group
0    10000032         2180       2014 - 2016
1    10000048         2126       2008 - 2010
2    10000068         2160       2008 - 2010
3    10000084         2160       2017 - 2019
4    10000102         2136       2008 - 2010


In [34]:
ddf_adm_pat_merged = ddf_admissions_mimiciv.merge(ddf_patients_mimiciv, how='inner', on='subject_id')
print(ddf_adm_pat_merged)

        subject_id deathtime  hospital_expire_flag  anchor_year  \
0         10467237       NaT                     0         2159   
1         16925328       NaT                     0         2143   
2         19430048       NaT                     0         2172   
3         18910522       NaT                     0         2142   
4         11146739       NaT                     0         2188   
...            ...       ...                   ...          ...   
431226    10382924       NaT                     0         2170   
431227    11326722       NaT                     0         2160   
431228    12133002       NaT                     0         2166   
431229    16572164       NaT                     0         2180   
431230    16549335       NaT                     0         2164   

       anchor_year_group  
0            2011 - 2013  
1            2008 - 2010  
2            2017 - 2019  
3            2011 - 2013  
4            2011 - 2013  
...                  ...  
431226

In [35]:
ddf_adm_pat_merged['median_anchor_year'] = ddf_adm_pat_merged['anchor_year_group'].str.split('-').apply(lambda x: int((int(x[0]) + int(x[1])) / 2))
ddf_adm_pat_merged['diff_year'] = ddf_adm_pat_merged['anchor_year'] - ddf_adm_pat_merged['median_anchor_year']
print(ddf_adm_pat_merged[['median_anchor_year', 'diff_year']])

        median_anchor_year  diff_year
0                     2012        147
1                     2009        134
2                     2018        154
3                     2012        130
4                     2012        176
...                    ...        ...
431226                2009        161
431227                2009        151
431228                2009        157
431229                2015        165
431230                2018        146

[431231 rows x 2 columns]


In [56]:
ddf_adm_pat_merged['month_of_death'] = ddf_adm_pat_merged['deathtime'].dt.month
ddf_adm_pat_merged['day_of_death'] = ddf_adm_pat_merged['deathtime'].dt.day

mask = (ddf_adm_pat_merged['hospital_expire_flag'] == 1) & (ddf_adm_pat_merged['deathtime'].notnull())

ddf_adm_pat_merged['year_of_death'] = ddf_adm_pat_merged['deathtime'].dt.year
ddf_adm_pat_merged['year_of_death'] = ddf_adm_pat_merged['deathtime'].dt.year - ddf_adm_pat_merged['diff_year']
ddf_adm_pat_merged.loc[mask, 'year_of_death'] = ddf_adm_pat_merged['year_of_death']

def calculate_final_day(row):
    year = row['year_of_death']
    month = row['month_of_death']
    if pd.isnull(year) or pd.isnull(month):
        return None
    else:
        return min(row['day_of_death'], pd.Period(year=year, month=month, freq='M').days_in_month)

ddf_adm_pat_merged['day_of_death_final'] = ddf_adm_pat_merged.apply(calculate_final_day, axis=1)

ddf_adm_pat_merged['death_date'] = pd.to_datetime({
    'year': ddf_adm_pat_merged['year_of_death'],
    'month': ddf_adm_pat_merged['month_of_death'],
    'day': ddf_adm_pat_merged['day_of_death_final']
})

ddf_adm_pat_merged['time_of_death'] = ddf_adm_pat_merged['deathtime'].dt.time
ddf_adm_pat_merged['time_of_death_str'] = ddf_adm_pat_merged['deathtime'].dt.strftime('%H:%M:%S') if 'deathtime' in ddf_adm_pat_merged.columns else ddf_adm_pat_merged['time_of_death']
ddf_adm_pat_merged['death_datetime'] = pd.NaT
ddf_adm_pat_merged['death_datetime'] = ddf_adm_pat_merged['death_datetime'].apply(lambda x: None if pd.isnull(x) else x)
valid_rows = ddf_adm_pat_merged.dropna(subset=['year_of_death', 'month_of_death', 'day_of_death', 'time_of_death_str'])
if not valid_rows.empty:
    ddf_adm_pat_merged.loc[valid_rows.index, 'death_datetime'] = pd.to_datetime(
        valid_rows[['year_of_death', 'month_of_death', 'day_of_death']].astype(int).astype(str).agg('-'.join, axis=1)
        + ' ' +
        valid_rows['time_of_death_str'],
        format='%Y-%m-%d %H:%M:%S',
        errors='coerce'
)
    
print(ddf_adm_pat_merged[['deathtime', 'diff_year', 'year_of_death', 'month_of_death', 'day_of_death', 'death_date', 'death_datetime']].iloc[:10])


            deathtime  diff_year  year_of_death  month_of_death  day_of_death  \
0                 NaT        147            NaN             NaN           NaN   
1                 NaT        134            NaN             NaN           NaN   
2                 NaT        154            NaN             NaN           NaN   
3                 NaT        130            NaN             NaN           NaN   
4                 NaT        176            NaN             NaN           NaN   
5                 NaT        116            NaN             NaN           NaN   
6 2150-03-03 09:21:00        133         2017.0             3.0           3.0   
7                 NaT        126            NaN             NaN           NaN   
8                 NaT        103            NaN             NaN           NaN   
9                 NaT        187            NaN             NaN           NaN   

  death_date       death_datetime  
0        NaT                 None  
1        NaT                 None  


In [57]:
ddf_adm_pat_merged['death_type_concept_id'] = 32817
print(ddf_adm_pat_merged[['death_type_concept_id']])

        death_type_concept_id
0                       32817
1                       32817
2                       32817
3                       32817
4                       32817
...                       ...
431226                  32817
431227                  32817
431228                  32817
431229                  32817
431230                  32817

[431231 rows x 1 columns]


In [62]:
ddf_adm_pat_merged['cause_concept_id'] = None
ddf_adm_pat_merged['cause_source_value'] = None
ddf_adm_pat_merged['cause_source_concept_id'] = None
print(ddf_adm_pat_merged[['cause_concept_id', 'cause_source_value', 'cause_source_concept_id']])

       cause_concept_id cause_source_value cause_source_concept_id
0                  None               None                    None
1                  None               None                    None
2                  None               None                    None
3                  None               None                    None
4                  None               None                    None
...                 ...                ...                     ...
431226             None               None                    None
431227             None               None                    None
431228             None               None                    None
431229             None               None                    None
431230             None               None                    None

[431231 rows x 3 columns]


In [67]:
ddf_adm_pat_merged_copy =  ddf_adm_pat_merged.copy()
ddf_adm_pat_merged_copy

,subject_id,deathtime,hospital_expire_flag,anchor_year,anchor_year_group,median_anchor_year,diff_year,month_of_death,day_of_death,time_of_death,year_of_death,time_of_death_str,death_datetime,death_year,day_of_death_final,death_date,death_type_concept_id,cause_concept_id,cause_source_value,cause_source_concept_id
0,10467237,NaT,0,2159,2011 - 2013,2012,147,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
1,16925328,NaT,0,2143,2008 - 2010,2009,134,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
2,19430048,NaT,0,2172,2017 - 2019,2018,154,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
3,18910522,NaT,0,2142,2011 - 2013,2012,130,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
4,11146739,NaT,0,2188,2011 - 2013,2012,176,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
431226,10382924,NaT,0,2170,2008 - 2010,2009,161,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
431227,11326722,NaT,0,2160,2008 - 2010,2009,151,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
431228,12133002,NaT,0,2166,2008 - 2010,2009,157,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None
431229,16572164,NaT,0,2180,2014 - 2016,2015,165,NaN,NaN,NaT,NaN,NaN,None,NaN,NaN,NaT,32817,None,None,None


In [68]:
!pip install --user pymysql
import pymysql
import MySQLdb

myvars = {}
with open("ksunchu-password") as myfile:
 for line in myfile:
    name, var = line.partition("=")[::2]
    myvars[name.strip()] = var.strip()

conn = MySQLdb.connect(host="127.0.0.1", user=myvars['username'], passwd=myvars['password'], db='omop')

omopcursor = conn.cursor()
print(conn)

<_mysql.connection open to '127.0.0.1' at 0x55a65360cb50>


In [71]:
death_data = ddf_adm_pat_merged_copy[['subject_id', 'death_date', 'death_datetime', 'death_type_concept_id', 'cause_concept_id', 'cause_source_value', 'cause_source_concept_id']]

for index, row in death_data.iterrows():
    row = row.where(pd.notnull(row), None)
    
    sql = "INSERT INTO DEATH (person_id, death_date, death_datetime, death_type_concept_id, cause_concept_id, cause_source_value, cause_source_concept_id) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    
    omopcursor.execute(sql, (
        row['subject_id'],
        row['death_date'],
        row['death_datetime'],
        row['death_type_concept_id'],
        row['cause_concept_id'],
        row['cause_source_value'],  # Fixed the missing closing quote
        row['cause_source_concept_id'],
    ))

conn.commit()
print("Insertion was successful.")


Insertion was successful.
